# Calculate Partial Derivatives of Phi to O2,T,S at O2=O2_bar, T=T_bar, S=S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'

### the partial derivatives

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_dPhidO2_dPhidT_dPhidS_1948-2017_{tres}.nc'
ds = xr.open_dataset(filename)

## dT, dS, dO2, dAOU

In [4]:
filename = f'{path}/DPLE-FOSI_hindcast_dO2_dT_dS_dAOU_1948-2017_{tres}.nc'
ds0 = xr.open_dataset(filename)
dT = ds0['dT']
dS = ds0['dS']
dO2 = ds0['dO2']
dAOU = ds0['dAOU']

## calculate Phi components

In [5]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [6]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product

for varr in ['O2', 'AOU', 'T', 'S']:
    dsg = xr.Dataset()
    for i, j in product(range(nrow), range(ncol)):
        Ac = Ac_dist[i]
        Eo = Eo_dist[j]
        print(Ac,Eo)
        if varr == 'O2':
            dsg[f'Phi_O2_{nrow*i+j}'] =  ds[f'dPhidO2_{nrow*i+j}'] * dO2 #dEodT=0.022
            print('Phi O2 done')
        elif varr == 'AOU':
            dsg[f'Phi_AOU_{nrow*i+j}'] = -ds[f'dPhidO2_{nrow*i+j}'] * dAOU
            print('Phi AOU done')
        elif varr == 'T':
            dsg[f'Phi_T_{nrow*i+j}'] = ds[f'dPhidT_{nrow*i+j}'] * dT  #dEodT=0.022
            print('Phi T done')
        elif varr == 'S':
            dsg[f'Phi_S_{nrow*i+j}'] = ds[f'dPhidS_{nrow*i+j}'] * dS  #dEodT=0.022
            print('Phi S done')
    dout = f'/glade/scratch/{USER}/DPLE-results'
    os.makedirs(dout, exist_ok=True)
    dsg.load()
    #
    dsg.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_1948-2017_{tres}.nc', mode='w')

10 -0.2
Phi O2 done
10 -0.1
Phi O2 done
10 0.0
Phi O2 done
10 0.10000000000000003
Phi O2 done
10 0.2
Phi O2 done
10 0.3
Phi O2 done
10 0.4000000000000001
Phi O2 done
10 0.5
Phi O2 done
10 0.6000000000000001
Phi O2 done
10 0.7
Phi O2 done
10 0.8
Phi O2 done
10 0.9000000000000001
Phi O2 done
10 1.0000000000000002
Phi O2 done
10 -0.2
Phi AOU done
10 -0.1
Phi AOU done
10 0.0
Phi AOU done
10 0.10000000000000003
Phi AOU done
10 0.2
Phi AOU done
10 0.3
Phi AOU done
10 0.4000000000000001
Phi AOU done
10 0.5
Phi AOU done
10 0.6000000000000001
Phi AOU done
10 0.7
Phi AOU done
10 0.8
Phi AOU done
10 0.9000000000000001
Phi AOU done
10 1.0000000000000002
Phi AOU done
10 -0.2
Phi T done
10 -0.1
Phi T done
10 0.0
Phi T done
10 0.10000000000000003
Phi T done
10 0.2
Phi T done
10 0.3
Phi T done
10 0.4000000000000001
Phi T done
10 0.5
Phi T done
10 0.6000000000000001
Phi T done
10 0.7
Phi T done
10 0.8
Phi T done
10 0.9000000000000001
Phi T done
10 1.0000000000000002
Phi T done
10 -0.2
Phi S done
10 -0.